In [99]:
import fitz
import pandas as pd
import numpy as np
import requests
import json
from fuzzywuzzy import process
import re


In [93]:
def extract_all_tables(pdf_path):
    tables = []
    doc = fitz.open(pdf_path)

    for page_number, page in enumerate(doc, 1):
        table_data = page.find_tables()
        if table_data.tables:
            for table in table_data.tables:
                raw_data = table.extract()
                if raw_data and len(raw_data):
                    df = pd.DataFrame(raw_data)
                    tables.append(df)
    
    if not tables:
        print("No tables found in the PDF.")
        return None
    
    result_df = pd.concat(tables, ignore_index=True)
    result_df = result_df.dropna(how='all')  # drop empty rows
    return result_df

# Example usage
pdf_path = r"D:\Mridul.Intern\OneDrive - Vivriti Capital Private Limited\Desktop\proj\desc ner\data\pdf\canara_esskay.pdf"

all_tables = extract_all_tables(pdf_path)


if all_tables is not None:
    # print(all_tables.head())
    print("\nTotal rows extracted:", len(all_tables))


table_text = all_tables.to_string(index=False)
all_tables


Total rows extracted: 26


,0,1,2,3,4,5,6,7
0,Account Holders Name,ESSKAY WIRE AND CABLE PRIVATE LIMITED,NaN,NaN,NaN,NaN,NaN,NaN
1,Customer Id,308144745,NaN,NaN,NaN,NaN,NaN,NaN
2,Branch Name,BAWANA,NaN,NaN,NaN,NaN,NaN,NaN
3,MICR Code,110015179,NaN,NaN,NaN,NaN,NaN,NaN
4,IFSC Code,CNRB0002972,NaN,NaN,NaN,NaN,NaN,NaN
5,Searched By,From 01 Jun 2024 To 30 Jun 2024,NaN,NaN,NaN,NaN,NaN,NaN
6,Account Number,120000785121,NaN,NaN,NaN,NaN,NaN,NaN
7,Account Currency,INR,NaN,NaN,NaN,NaN,NaN,NaN
8,Product Name,CANARA PRIVILEGE,NaN,NaN,NaN,NaN,NaN,NaN
9,Opening Balance,"Rs. 3,506.11",NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
#desc cleaning

# transaction types
TRANSACTION_TYPES = {
    'NEFT', 'RTGS', 'POS', 'ACH', 'IMPS', 'UPI',
    'NACH', 'FT', 'DD', 'ECS', 'AEPS', 'SWIFT'
}

# pre-compile regex patterns
ACCOUNT_NO_PATTERN = re.compile(r'([A-Z]{4,}\d{6,}\w*)')
ALPHA_NUMERIC_WORD_PATTERN = re.compile(r'\b(?=\w*[A-Za-z])(?=\w*\d)\w+\b')
TRANSACTION_TYPE_PATTERN = re.compile(r'\b(?:' + '|'.join(TRANSACTION_TYPES) + r')\b', re.IGNORECASE)
NON_ALPHA_SPACE_PATTERN = re.compile(r'[^A-Za-z\s]')
MONTHS_REMOVE = re.compile(r'(?i)\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|sept?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b')

STOP_WORDS = {
    'CR', 'DR', 'BY', 'TO', 'FROM', 'TRANSFER', 'PAYMENT',
    'CREDIT', 'DEBIT', 'THROUGH', 'VIA', 'TRANSACTION', 'CHQ', 
    'ADV', 'CHEQUE', 'DEPOSIT', 'OUTWARD', 'INWARD'
}

def should_process(text):
    """Check if text contains any of the transaction types"""
    if pd.isna(text) or not isinstance(text, str):
        return False

    return any(trans_type in text.upper() for trans_type in TRANSACTION_TYPES)

def extract_after_account_no(text):
    """
    Detect account number and return everything after it.
    """
    match = ACCOUNT_NO_PATTERN.search(text)
    if match:
        return text[match.end():].strip()
    return text

def clean_text(text):
    """Clean bank statement description text using regex"""
    if not should_process(text):
        return ""

    # 1. Extract everything after the account number
    processed_text = extract_after_account_no(text)

    # 2. Remove any group that contains both letters and digits (alphanumeric words)
    processed_text = ALPHA_NUMERIC_WORD_PATTERN.sub('', processed_text)

    # 3. Remove transaction type keywords (case-insensitive, as whole words)
    processed_text = TRANSACTION_TYPE_PATTERN.sub('', processed_text)

    # 4. Remove abbreviated months and dates
    processed_text = MONTHS_REMOVE.sub('', processed_text)

    # 5. Remove special characters except alphabets and spaces, convert to uppercase, and collapse spaces
    processed_text = NON_ALPHA_SPACE_PATTERN.sub(' ', processed_text)
    processed_text = processed_text.upper()
    processed_text = ' '.join(processed_text.split())

    # 6. Remove common transaction-related words
    words = [w for w in processed_text.split() if w not in STOP_WORDS]

    return ' '.join(words)


In [110]:
def extract_transactions(df):
    # Find the header row index
    header_row_idx = None
    for i, row in df.iterrows():
        if any("txn date" in str(cell).lower() for cell in row):
            header_row_idx = i
            break
    if header_row_idx is None:
        raise ValueError("Transaction header row not found.")

    # Set new header and get transaction data
    new_header = df.iloc[header_row_idx]
    df_txn = df.iloc[header_row_idx+1:].copy()
    df_txn.columns = new_header
    df_txn = df_txn.reset_index(drop=True)
    return df_txn

def standardize(df):
    df = df.loc[:, ~df.columns.duplicated()]
    df.columns = df.columns.astype(str)
    df.columns = (df.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(r'\s+', '_', regex=True)
                  .str.replace(r'\.', '', regex=True))
    standard_cols = ['date', 'description', 'debit', 'credit', 'balance']
    matched_cols = {}
    used_cols = set()
    for std_col in standard_cols:
        match, score = process.extractOne(std_col, df.columns)
        if score > 60 and match not in used_cols:
            matched_cols[std_col] = match
            used_cols.add(match)
        else:
            matched_cols[std_col] = None
    std_df = pd.DataFrame()
    for std_col in standard_cols:
        if matched_cols[std_col]:
            std_df[std_col] = df[matched_cols[std_col]]
        else:
            std_df[std_col] = ""
    return std_df

def clean_repeated_headers(df):
    # Remove rows where 'Txn Date' (or similar) appears again in any column
    mask = df.apply(lambda row: row.astype(str).str.lower().str.contains("txn date").any(), axis=1)
    df_clean = df[~mask].copy()
    # Drop fully empty rows
    df_clean = df_clean.dropna(how='all')
    # Shift up rows if there are empty rows (optional, usually dropna is enough)
    df_clean = df_clean.reset_index(drop=True)
    return df_clean

def clean_description(df):
    """Apply clean_text to the 'description' column and add a new column 'cleaned_description'."""
    if 'description' in df.columns:
        df['description'] = df['description'].apply(clean_text)
    else:
        df['description'] = ""
    return df

def calculate_metrics(df):
    """Calculate total debit, credit, opening and closing balance"""
    total_credit = 0
    total_debit = 0
    opening_bal = 0
    closing_bal = 0

    if 'date' in df.columns and df['date'].notnull().any():
        if 'debit' in df.columns:
                debit_str = df['debit'].astype(str).replace('nan', '').str.replace(',', '', regex=False).str.strip()
                debit_str = debit_str.replace('', '0')
                total_debit = pd.to_numeric(debit_str, errors='coerce').sum()
        if 'credit' in df.columns:
                credit_str = df['credit'].astype(str).replace('nan', '').str.replace(',', '', regex=False).str.strip()
                credit_str = credit_str.replace('', '0')
                total_credit = pd.to_numeric(credit_str, errors='coerce').sum()
        if 'balance' in df.columns:
            balances = pd.to_numeric(df['balance'].replace('', '0').str.replace(',', '', regex=False), errors='coerce')
            opening_bal = balances.iloc[0] if not balances.empty else 0
            closing_bal = balances.iloc[-1] if not balances.empty else 0
    else:
        print("No debit/credit rows found")
        pass

    print("Total credit :", total_credit)
    print("Total debit :", total_debit)
    print("Opening balance :", opening_bal)
    print("Closing balance :", closing_bal)
    print(total_credit - total_debit)
    print(closing_bal - opening_bal)

if all_tables is not None:
    txn_df = extract_transactions(all_tables)
    txn_df = clean_repeated_headers(txn_df)
    std_table = standardize(txn_df)
    # td_table = clean_description(std_table)
    calculate_metrics(std_table)
    std_table.to_excel('output.xlsx', index=False)
    # table_text = std_table.head(50).to_string(index=False)

txn_df

Total credit : 4610000.0
Total debit : 4605352.0
Opening balance : 478506.11
Closing balance : 8154.11
4648.0
-470352.0


11,Txn Date,Value Date,Cheque No.,Description,Branch\nCode,Debit,Credit,Balance
0,04-06-2024 18:45:49,04 Jun 2024,,RTGS Cr-KKBKR12024060400894761-\nKKBK0000958-E...,33,,"4,75,000.00","4,78,506.11"
1,05-06-2024 13:14:20,05 Jun 2024,000000000000,NACH PROFECTUSCAPITALPR\nPCPL050624703\nCNRB00...,2972,"4,70,504.00",,"8,002.11"
2,11-06-2024 18:45:38,11 Jun 2024,416318684993,INET-IMPS-CR/ESSKAY WIR/\nKMB /3047662835/IB:\...,33,,"10,000.00","18,002.11"
3,11-06-2024 18:59:58,11 Jun 2024,,IB ITG 6244600 Online Transaction BDP-\nESI-10...,2972,"3,695.00",,"14,307.11"
4,13-06-2024 11:02:37,13 Jun 2024,,IB ITG 1510000 Online Transaction BDP-\nESI-10...,2972,"3,698.00",,"10,609.11"
5,21-06-2024 17:53:42,21 Jun 2024,,IB NEFT Dr P173240330762814 PAHAL\nFIRE ENGINE...,2972,"1,062.00",,"9,547.11"
6,21-06-2024 17:53:42,21 Jun 2024,,SC NEFT OTHER THAN SB IMB,2972,3.00,,"9,544.11"
7,23-06-2024 22:46:12,23 Jun 2024,,SMS CHARGES ON ACTUAL BASIS,2972,4.00,,"9,540.11"
8,28-06-2024 14:27:50,28 Jun 2024,,RTGS Cr-KKBKR12024062800868516-\nKKBK0000958-E...,33,,"41,25,000.00","41,34,540.11"
9,28-06-2024 15:09:46,28 Jun 2024,000460569532,RTGS Dr-CNRBR52024062883363267-\nIDIB000A687-R...,2972,"41,25,000.00",,"9,540.11"


In [95]:
import numpy as np
std_table['debit'] = (std_table['debit'].replace(',', '', regex=True).replace(['', ' '], np.nan).astype(float))
std_table['credit'] = (std_table['credit'].replace(',', '', regex=True).replace(['', ' '], np.nan).astype(float))

std_table['debit'].sum()

np.float64(4605352.0)

In [96]:
std_table['credit'].sum()

np.float64(4610000.0)

In [97]:
prompt = f"""You are given a CSV extracted from a bank statement. The structure is defined as :
1. Date of transaction
2. Transaction description
3. Credit amount (money received)
4. Debit amount (money spent)
5. Running balance

Perform the following tasks, and output only the following items:
1. Total Credit, simply add all amounts under the credit column
2. Total Debit, simply add all amounts under the debit column
3. Time Period: Earliest and Latest Transaction Dates
4. Top 5 Creditors (by credit amount) , including names (from the description) and total amount
5. Top 5 Debitors (by debit amount), including names (from the description) and total amount
6. Opening Balance (from the first transaction) and Closing Balance (from the last transaction)

Please keep in mind the follwing mpints :
Each row will only have one of debit or credit value.
The description column contains type of transaction followed by alphanumeric which is the account number and then possible an entity name which should be the name of the benificary, which would tell to whom the money is being sent to or being recieved from.
The description column will also contain transaction comments towards the end.
If any required value is missing or unidentifiable, state “Unavailable” in its place.

For example : 02-05-2022 11:35:03;ECS TPCapfrstIDFCFIRST;36,621.00;;4,49,957.71
Here the transaction is ECS, benificary name is IDFCFIRST, debit amount is 36,621 and the balance after the transaction is 4,49,957.71

Input CSV:
{table_text}
"""

url = "http://localhost:11434/api/chat"
payload = {
    "model": "deepseek-r1",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature" : 0
}

response = requests.post(url, json=payload, stream=True)

for line in response.iter_lines():
    if line:
        data = line.decode("utf-8")
        import json
        chunk = json.loads(data)
        if "message" in chunk and "content" in chunk["message"]:
            print(chunk["message"]["content"], end="", flush=True)
print()  # for newline

<think>
Okay, so I need to help this user with analyzing their bank statement CSV data. They want me to extract several pieces of information without any extra output—just the total credit and debit amounts, the time period from earliest to latest transaction dates, top five creditors by credit amount including names and totals, top five debitors similarly, and both the opening and closing balances.

First, I'll start by understanding the CSV structure. Each row has a date of transaction, description, debit amount, credit amount, and running balance. The key here is that each transaction either credits or debits money but not both in the same line, which simplifies some calculations because we don't have to worry about transactions with no value.

Looking at the example they provided: "02-05-2022 11:35:03;ECS TPCapfrstIDFCFIRST;36,621.00;;4,49,957.71" shows a typical transaction line where the description includes details like "ECS", and then it specifies the beneficiary name as "IDFCF

KeyboardInterrupt: 

In [4]:
from PIL import Image
from pdf2image import convert_from_path

def pdf_to_img(pdf_path, output_folder, poppler_path):
    images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
    for i, img in enumerate(images):
        output_path = f"{output_folder}/page_{i+1}.png"
        img.save(output_path, 'PNG')
        print(f"Saved: {output_path}")

pdf_path = r"D:\Mridul.Intern\OneDrive - Vivriti Capital Private Limited\Desktop\proj\desc ner\data\old pdfs\axis_aditya.pdf"
output_folder = r"images"
poppler_bin = r"D:\\Mridul.Intern\\poppler-23.05.0\\Library\\bin"
pdf_to_img(pdf_path, output_folder,poppler_bin)

Saved: images/page_1.png
Saved: images/page_2.png
Saved: images/page_3.png
Saved: images/page_4.png


In [ ]:
import base64
import os
import requests

def encode_images_to_base64(image_folder):
    images_b64 = []
    for filename in sorted(os.listdir(image_folder)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            with open(os.path.join(image_folder, filename), "rb") as img_file:
                b64_str = base64.b64encode(img_file.read()).decode('utf-8')
                images_b64.append(b64_str)
    return images_b64

# Usage
image_folder = r"images"
images_b64 = encode_images_to_base64(image_folder)

prompt = f"""These images are extracted from a bank statement. I want to find the total amount credited and debited.
These amounts are below the respective names, and their total may also be found at the end of the statement for reference.
Do not use the last table with bank charges in the header.
The opening and closing balances are also mentioned.
"""

url = "http://localhost:11434/api/chat"
payload = {
    "model": "llava",
    "messages": [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{images_b64[0]}"}}
        ]
    }
    ],
    "images": images_b64,
    "temperature": 0
}

try:
    response = requests.post(url, json=payload, stream=True)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Request failed: {e}")
    exit()

for line in response.iter_lines():
    if line:
        data = line.decode("utf-8")
        import json
        chunk = json.loads(data)
        if "message" in chunk and "content" in chunk["message"]:
            print(chunk["message"]["content"], end="", flush=True)
print()  # for newline

Request failed: 400 Client Error: Bad Request for url: http://localhost:11434/api/chat



: 

In [13]:
prompt = f"""This is a CSV extracted from a bank statement. The description column is missing.
There is an additional table at the end, please remove it and return as json

Input CSV:{table_text}
"""

url = "http://localhost:11434/api/chat"
payload = {
    "model": "deepseek-r1",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "temperature" : 0
}

response = requests.post(url, json=payload, stream=True)

for line in response.iter_lines():
    if line:
        data = line.decode("utf-8")
        import json
        chunk = json.loads(data)
        if "message" in chunk and "content" in chunk["message"]:
            print(chunk["message"]["content"], end="", flush=True)
print()  # for newline

<think>
Alright, let's break down this problem. The user provided a list of transactions and charges for an account from December 2022 to March 2023. I need to understand the structure here.

First, looking at the transaction details:
- There are many entries with dates, charge types like Insurance Premium, Annual Service Fee, Renewal Processing Fee, etc.
- The closing balance is negative (-1817891.53), which suggests that there might be some overpayment or perhaps a loan repayment.

Now, looking at the Sr. No. table:
- Monthly Service Charge of ₹25 for each period from 06-2023 to 12-2022, except possibly February 2023 (marked as None). Wait, that seems off because the periods listed are from December 2022 onwards.

Wait, I notice some inconsistencies here. The Sr. No. shows charges for different months but the dates in the first part go up to May 2023. Maybe there's a mix-up between charge types and transaction fees?

Also, note that sometimes multiple entries exist for the same month